<a href="https://colab.research.google.com/github/neha11shete/AIML/blob/main/Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.22.1 datasets

In [1]:
from datasets import load_dataset
raw_datasets = load_dataset("squad")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
raw_datasets["train"][1]["title"]

In [ ]:
raw_datasets["train"][1]["context"]

In [ ]:
raw_datasets["train"][1]["question"]

In [ ]:
raw_datasets["train"][1]["answers"]

In [ ]:
# for train set, ensure that there's always 1 answer
# not multiple answers, or no answers
filtered_dataset = raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)
print(filtered_dataset) # num_rows: 0 means no a single answer list with mulitple answers in data set for train

In [ ]:
raw_datasets["validation"][2]["answers"]

In [ ]:
# why are there multiple answers?
raw_datasets["validation"][2]["context"]

In [ ]:
raw_datasets["validation"][2]["question"]

In [ ]:
# they may even be the same!
raw_datasets["validation"][0]["answers"]

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased"
# model_checkpoint = "bert-base-cased" # try it yourself
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
context = raw_datasets["train"][1]["context"]
question = raw_datasets["train"][1]["question"]

inputs = tokenizer(question, context)
print(inputs)
tokenizer.decode(inputs["input_ids"]) #decoded back from input

In [ ]:
# what if the context is really long?
# split it into multiple samples
inputs = tokenizer(
  question,
  context,
  max_length=384,
  truncation="only_second", #only context is to be truncated
  stride=128, #overalapping context windows
  return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

print(inputs['overflow_to_sample_mapping']) #same context in 4 windows divided, 0 is for row number (1st data sample, it will be different as sample size increases)

In [ ]:
inputs.keys()

In [ ]:
inputs = tokenizer(
    raw_datasets["train"][:3]["question"], #first 3 sample data
    raw_datasets["train"][:3]["context"],
    max_length=384,
    truncation="only_second",
    stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs['overflow_to_sample_mapping'] #4 context windows for sample data at position 0, 4 context window for sample data at position 1 etc

In [ ]:
inputs

In [ ]:
inputs.keys()

In [ ]:
# what is this (weirdly named) offset_mapping?
# it tells us the location of each token
# notes:
# special tokens take up 0 space - (0, 0)
# the question portion is the same for each sample
# the context portion starting point inceases in each sample
inputs['offset_mapping'][0] #in same array (0, 0)like CLS and SEP... first start of (0,0)that till 71 is the question,remaining is the 1st context window length



In [ ]:
raw_datasets["train"][0]['question'] # To 2 (0,2) Whom 4 (3,7)

In [ ]:
print(len(raw_datasets["train"][0]['question'])) #for context window its counting start from 00 of SEP

In [ ]:
print(len(inputs["input_ids"][0]))

In [ ]:
print(inputs.sequence_ids(0))
print(len(inputs.sequence_ids(0))) #0 its question after that all 1 are context
print(len(inputs['offset_mapping'][0]))

In [ ]:
print(inputs.sequence_ids(1))

In [ ]:
# find the start and end of the context (the first and last '1')
#sequence_ids = inputs.sequence_ids(4)
sequence_ids = inputs.sequence_ids(0)
ctx_start = sequence_ids.index(1) # first occurrence
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1 # last occurrence

ctx_start, ctx_end #sequence id

In [ ]:
# check whether or not the answer is fully contained within the context
# if not, target is (start, end) = (0, 0)
answer =  raw_datasets["train"][0]["answers"]
#print(answer)
ans_start_char = answer['answer_start'][0] #515
ans_end_char = ans_start_char + len(answer['text'][0]) #541
#print(ans_start_char,ans_end_char)
#print(ctx_start,ctx_end,inputs['offset_mapping'][2])

offset = inputs['offset_mapping'][2]

start_idx = 0
end_idx = 0
#print(offset[ctx_start],offset[ctx_end])
print(offset[ctx_start][0],offset[ctx_end][1])
if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
  print("target is (0, 0)")
  # nothing else to do
else:
  # find the start and end TOKEN positions

  # the 'trick' is knowing what is in units of tokens and what is in
  # units of characters

  # recall: the offset_mapping contains the character positions of each token
   #eg 500 to 541 context 515 to 550
  i = ctx_start #500
  for start_end_char in offset[ctx_start:]:
    start, end = start_end_char
    print(start,ans_start_char,end,ans_end_char)
    if start == ans_start_char:
      start_idx = i #515
      # don't break yet

    if end == ans_end_char:
      end_idx = i
      break

    i += 1

start_idx, end_idx

In [ ]:
# check
input_ids = inputs['input_ids'][2]
print(input_ids[ans_start_char : ans_end_char + 1],ans_start_char,ans_end_char)
print(input_ids[start_idx : end_idx + 1],start_idx)

In [ ]:
tokenizer.decode(input_ids[start_idx : end_idx + 1])

In [ ]:
def find_answer_token_idx(
    ctx_start,
    ctx_end,
    ans_start_char,
    ans_end_char,
    offset):

  start_idx = 0
  end_idx = 0

  if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
    pass
    # print("target is (0, 0)")
    # nothing else to do
  else:
    # find the start and end TOKEN positions

    # the 'trick' is knowing what is in units of tokens and what is in
    # units of characters

    # recall: the offset_mapping contains the character positions of each token

    i = ctx_start
    for start_end_char in offset[ctx_start:]:
      start, end = start_end_char
      if start == ans_start_char:
        start_idx = i
        # don't break yet

      if end == ans_end_char:
        end_idx = i
        break

      i += 1
  return start_idx, end_idx

In [ ]:
# try it on all context windows
# sometimes, the answer won't appear!

start_idxs = []
end_idxs = []

for i, offset in enumerate(inputs['offset_mapping']):
  # the final window may not be full size - can't assume 100
  sequence_ids = inputs.sequence_ids(i)
  print("sequence_ids ",i)
  # find start + end of context (first 1 and last 1)
  ctx_start = sequence_ids.index(1)
  ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

  start_idx, end_idx = find_answer_token_idx(
    ctx_start,
    ctx_end,
    ans_start_char,
    ans_end_char,
    offset)

  start_idxs.append(start_idx)
  end_idxs.append(end_idx)

start_idxs, end_idxs #All context of input have been listen below, since context are overlapping Saint Bernadette Soubirous might be part of two context as here 72 to 78 in 2 and 40 to 46 in 3

In [ ]:
for q in raw_datasets["train"]["question"][:1000]:
  if q.strip() != q:
    print(q)

In [ ]:
# now we are ready to process (tokenize) the training data
# (i.e. expand question+context pairs into question+smaller context windows)

# Google used 384 for SQuAD
max_length = 384
stride = 128

def tokenize_fn_train(batch):
  # some questions have leading and/or trailing whitespace
  questions = [q.strip() for q in batch["question"]]

  # tokenize the data (with padding this time)
  # since most contexts are long, we won't bother to pad per-minibatch
  inputs = tokenizer(
    questions,
    batch["context"],
    max_length=max_length,
    truncation="only_second",
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )

  # we don't need these later so remove them
  offset_mapping = inputs.pop("offset_mapping")
  orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
  answers = batch['answers']
  answers
  start_idxs, end_idxs = [], []

  # same loop as above
  for i, offset in enumerate(offset_mapping):
    sample_idx = orig_sample_idxs[i]
    answer = answers[sample_idx]

    ans_start_char = answer['answer_start'][0]
    ans_end_char = ans_start_char + len(answer['text'][0])

    sequence_ids = inputs.sequence_ids(i)

    # find start + end of context (first 1 and last 1)
    ctx_start = sequence_ids.index(1)
    ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

    start_idx, end_idx = find_answer_token_idx(
      ctx_start,
      ctx_end,
      ans_start_char,
      ans_end_char,
      offset)

    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

  inputs["start_positions"] = start_idxs
  inputs["end_positions"] = end_idxs
  return inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
  tokenize_fn_train,
  batched=True,
  remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

In [ ]:
train_dataset

In [ ]:
train_dataset[0]['start_positions']

In [ ]:
train_dataset[0]['end_positions']

In [ ]:
# tokenize the validation set differently
# we won't need the targets since we will just compare with the original answer
# also: overwrite offset_mapping with Nones in place of question
def tokenize_fn_validation(batch):
  # some questions have leading and/or trailing whitespace
  questions = [q.strip() for q in batch["question"]]

  # tokenize the data (with padding this time)
  # since most contexts are long, we won't bother to pad per-minibatch
  inputs = tokenizer(
    questions,
    batch["context"],
    max_length=max_length,
    truncation="only_second",
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )

  # we don't need these later so remove them
  orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
  print(orig_sample_idxs)
  sample_ids = []

  # rewrite offset mapping by replacing question tuples with None
  # this will be helpful later on when we compute metrics
  for i in range(len(inputs["input_ids"])):
    sample_idx = orig_sample_idxs[i]
    print(i,batch['id'][sample_idx],sample_idx) #if [0000] i =1 , but for sample_idx will be zero because of context window
    sample_ids.append(batch['id'][sample_idx]) #batch id

    sequence_ids = inputs.sequence_ids(i)
    offset = inputs["offset_mapping"][i]
    inputs["offset_mapping"][i] = [
      x if sequence_ids[j] == 1 else None for j, x in enumerate(offset)]

  inputs['sample_id'] = sample_ids
  return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
  tokenize_fn_validation,
  batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

In [ ]:
from datasets import load_metric

metric = load_metric("squad")

In [ ]:
predicted_answers = [
  {'id': '1', 'prediction_text': 'Albert Einstein'},
  {'id': '2', 'prediction_text': 'physicist'},
  {'id': '3', 'prediction_text': 'general relativity'},
]
true_answers = [
  {'id': '1', 'answers': {'text': ['Albert Einstein'], 'answer_start': [100]}},
  {'id': '2', 'answers': {'text': ['physicist'], 'answer_start': [100]}},
  {'id': '3', 'answers': {'text': ['special relativity'], 'answer_start': [100]}},
]

# id and answer_start seem superfluous but you'll get an error if not included
# exercise: remove them (one at a time) and see!
metric.compute(predictions=predicted_answers, references=true_answers)

In [ ]:
# next problem: how to go from logits to prediction text?
# to make it easier, let's work on an already-trained question-answering model
small_validation_dataset = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer2 = AutoTokenizer.from_pretrained(trained_checkpoint)

# temporarily assign tokenizer2 to tokenizer since it's used as a global
# in tokenize_fn_validation
old_tokenizer = tokenizer
tokenizer = tokenizer2

small_validation_processed = small_validation_dataset.map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

# change it back
tokenizer = old_tokenizer

In [ ]:
# get the model outputs
import torch
from transformers import AutoModelForQuestionAnswering

# the trained model doesn't use these columns
small_model_inputs = small_validation_processed.remove_columns(
  ["sample_id", "offset_mapping"])
small_model_inputs.set_format("torch")

# get gpu device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# move tensors to gpu device
small_model_inputs_gpu = {
  k: small_model_inputs[k].to(device) for k in small_model_inputs.column_names
}

# download the model
trained_model = AutoModelForQuestionAnswering.from_pretrained(
  trained_checkpoint).to(device)

# get the model outputs
with torch.no_grad():
  outputs = trained_model(**small_model_inputs_gpu)

In [ ]:
outputs

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
small_validation_processed['sample_id'][:5]

In [ ]:
# example: {'56be4db0acb8001400a502ec': [0, 1, 2, 3], ...}
sample_id2idxs = {}
for i, id_ in enumerate(small_validation_processed['sample_id']):
  if id_ not in sample_id2idxs:
    sample_id2idxs[id_] = [i]
  else:
    print("here")
    sample_id2idxs[id_].append(i)

In [ ]:
start_logits.shape, end_logits.shape

In [ ]:
# reminder of how to find indices with the largest values
(-start_logits[0]).argsort()

In [ ]:
n_largest = 20
max_answer_length = 30
predicted_answers = []

# we are looping through the original (untokenized) dataset
# because we need to grab the answer from the original string context
for sample in small_validation_dataset:
  sample_id = sample['id']
  context = sample['context']

  # update these as we loop through candidate answers
  best_score = float('-inf')
  best_answer = None

  # now loop through the *expanded* input samples (fixed size context windows)
  # from here we will pick the highest probability start/end combination
  for idx in sample_id2idxs[sample_id]:
    start_logit = start_logits[idx] # (384,) vector
    end_logit = end_logits[idx] # (384,) vector
    offsets = small_validation_processed[idx]['offset_mapping']

    start_indices = (-start_logit).argsort() #indexes with largest value sorted in desc
    end_indices = (-end_logit).argsort()

    for start_idx in start_indices[:n_largest]: #first 20
      for end_idx in end_indices[:n_largest]:#first 20

        # skip answers not contained in context window
        # recall: we set entries not pertaining to context to None earlier
        if offsets[start_idx] is None or offsets[end_idx] is None:
          continue

        # skip answers where end < start
        if end_idx < start_idx:
          continue

        # skip answers that are too long
        if end_idx - start_idx + 1 > max_answer_length:
          continue

        # see theory lecture for score calculation
        score = start_logit[start_idx] + end_logit[end_idx]
        if score > best_score:
          best_score = score

          # find positions of start and end characters
          # recall: offsets contains tuples for each token:
          # (start_char, end_char)
          first_ch = offsets[start_idx][0]
          last_ch = offsets[end_idx][1]

          best_answer = context[first_ch:last_ch]

  # save best answer
  predicted_answers.append({'id': sample_id, 'prediction_text': best_answer})

In [ ]:
# now test it!

true_answers = [
  {'id': x['id'], 'answers': x['answers']} for x in small_validation_dataset
]
metric.compute(predictions=predicted_answers, references=true_answers)

In [ ]:
# now let's define a full compute_metrics function
# note: this will NOT be called from the trainer

from tqdm.autonotebook import tqdm

def compute_metrics(start_logits, end_logits, processed_dataset, orig_dataset):
  # map sample_id ('56be4db0acb8001400a502ec') to row indices of processed data
  sample_id2idxs = {}
  for i, id_ in enumerate(processed_dataset['sample_id']):
    if id_ not in sample_id2idxs:
      sample_id2idxs[id_] = [i]
    else:
      sample_id2idxs[id_].append(i)

  predicted_answers = []
  for sample in tqdm(orig_dataset):

    sample_id = sample['id']
    context = sample['context']

    # update these as we loop through candidate answers
    best_score = float('-inf')
    best_answer = None

    # now loop through the *expanded* input samples (fixed size context windows)
    # from here we will pick the highest probability start/end combination
    for idx in sample_id2idxs[sample_id]:
      start_logit = start_logits[idx] # (T,) vector
      end_logit = end_logits[idx] # (T,) vector

      # note: do NOT do the reverse: ['offset_mapping'][idx]
      offsets = processed_dataset[idx]['offset_mapping']

      start_indices = (-start_logit).argsort()
      end_indices = (-end_logit).argsort()

      for start_idx in start_indices[:n_largest]:
        for end_idx in end_indices[:n_largest]:

          # skip answers not contained in context window
          # recall: we set entries not pertaining to context to None earlier
          if offsets[start_idx] is None or offsets[end_idx] is None:
            continue

          # skip answers where end < start
          if end_idx < start_idx:
            continue

          # skip answers that are too long
          if end_idx - start_idx + 1 > max_answer_length:
            continue

          # see theory lecture for score calculation
          score = start_logit[start_idx] + end_logit[end_idx]
          if score > best_score:
            best_score = score

            # find positions of start and end characters
            # recall: offsets contains tuples for each token:
            # (start_char, end_char)
            first_ch = offsets[start_idx][0]
            last_ch = offsets[end_idx][1]

            best_answer = context[first_ch:last_ch]

    # save best answer
    predicted_answers.append({'id': sample_id, 'prediction_text': best_answer})

  # compute the metrics
  true_answers = [
    {'id': x['id'], 'answers': x['answers']} for x in orig_dataset
  ]
  return metric.compute(predictions=predicted_answers, references=true_answers)

In [ ]:
# run our function on the same mini dataset as before
compute_metrics(
    start_logits,
    end_logits,
    small_validation_processed,
    small_validation_dataset,
)

In [ ]:
# now load the model we want to fine-tune
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)

In [ ]:
from transformers import Trainer

# takes ~2.5h with bert on full dataset
# ~1h 15min with distilbert

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    # train_dataset=train_dataset.shuffle(seed=42).select(range(1_000)),
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
trainer_output = trainer.predict(validation_dataset)

In [ ]:
type(trainer_output)

In [ ]:
trainer_output

In [ ]:
predictions, _, _ = trainer_output

In [ ]:
predictions

In [ ]:
start_logits, end_logits = predictions

In [ ]:
compute_metrics(
    start_logits,
    end_logits,
    validation_dataset, # processed
    raw_datasets["validation"], # orig
)

In [ ]:
trainer.save_model('my_saved_model')

In [ ]:
from transformers import pipeline

qa = pipeline(
  "question-answering",
  model='my_saved_model',
  device=0,
)

In [ ]:
context = "Today I went to the store to purchase a carton of milk."
question = "What did I buy?"

qa(context=context, question=question)